In [1]:
import os
import json

import numpy as np
import torch
import torchvision
from torchsummary import summary

from project_18408.datasets import *
from project_18408.evaluation import *
from project_18408.experiments import *

In [2]:
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version: 1.8.0
Torchvision Version: 0.9.0
Using the GPU!


In [3]:
# fixed paramaters
img_dataset_type = ImageDatasetType.MNIST
layer_width = 100
num_epochs = 60
input_img_dim = IMG_DATASET_TO_IMG_SIZE[img_dataset_type]
input_num_channels = 1
output_dim = IMG_DATASET_TO_NUM_CLASSES[img_dataset_type]
lr = 0.01

# varied parameters
num_train_samples_space = np.array([20000, 40000, 60000])
depth_space = np.array([3, 5, 10, 15, 20, 30])
weight_decay_space = np.array([0, 1e-5, 1e-4])
label_corrupt_frac_space = np.array([0])
trial_index_space = np.array([0])

In [4]:
def generate_config(num_train_samples,
                    depth,
                    weight_decay,
                    corrupt_frac,
                    trial_index):
    num_train_samples = int(num_train_samples)
    depth = int(depth)
    weight_decay = float(weight_decay)
    corrupt_frac = float(corrupt_frac)
    trial_index = int(trial_index)
    early_stop_acc = (1-corrupt_frac) * 0.75
    
    img_dataset_type = ImageDatasetType.MNIST
    dataset_config = DatasetConfig(
        dataset_type=DatasetType.IMG,
        dataset_config=ImageDatasetConfig(
            img_dataset_type=img_dataset_type,
            num_train_samples=num_train_samples,
            corrupt_frac=corrupt_frac,
            flatten=False,
            seed=0
        )
    )

    model_config = ModelConfig(
        model_type=ModelType.CNN_TOY,
        model_config=CNNModelConfig(
             depth=depth, 
             input_img_dim=input_img_dim, 
             input_num_channels=input_num_channels,
             output_dim=output_dim, 
             kernel_size=3,
             seed=None
        )
    )

    training_config = TrainingConfig(
        optimizer_type=OptimizerType.SGD_MOMENTUM,
        loss_type=LossType.CROSS_ENTROPY,
        num_epochs=num_epochs,
        weight_decay=weight_decay,
        early_stop=True,
        early_stop_acc=early_stop_acc,
        lr=lr
    )

    experiment_config = ExperimentConfig(
        dataset_config=dataset_config,
        model_config=model_config,
        training_config=training_config,
        trial_index=trial_index
    )
    
    return experiment_config

In [5]:
experiment_configs = []

spaces = (num_train_samples_space, depth_space, weight_decay_space, label_corrupt_frac_space, trial_index_space)
grids = np.meshgrid(*spaces, indexing='ij')
for index in np.ndindex(grids[0].shape):
    params = [g[index] for g in grids]
    config = generate_config(*params)
    experiment_configs.append(config)

In [6]:
print(len(experiment_configs), "configurations generated")

63 configurations generated


In [7]:
configs_fname = "experiment_configs_mnist_test3.json"

In [8]:
out = [c.to_dict() for c in experiment_configs]
with open(configs_fname, 'w') as f:
    json.dump(out, f, indent=4)